In [1]:
import os
import pandas as pd
import numpy as np

## 载入数据到 Pandas

* 索引：将一个列或多个列读取出来构成 DataFrame，其中涉及是否从文件中读取索引以及列名
* 类型推断和数据转换：包括用户自定义的转换以及缺失值标记
* 日期解析
* 迭代：针对大文件进行逐块迭代。这个是Pandas和Python原生的csv库的最大区别
* 不规整数据问题：跳过一些行，或注释等等

## 索引及列名

In [3]:
os.chdir(r'/Users/joey/PycharmProjects/mywork')

In [52]:
%pwd

'/Users/joey/PycharmProjects/mywork'

In [53]:
%more Pandas/ex1.csv

In [2]:
# df = pd.read_csv('Pandas/ex1.csv')
df = pd.read_csv('Pandas/ex1.csv',sep=',')
df

FileNotFoundError: [Errno 2] File Pandas/ex1.csv does not exist: 'Pandas/ex1.csv'

In [55]:
%more Pandas/ex2.csv

In [56]:
# 列名缺失
pd.read_csv('Pandas/ex2.csv',header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [57]:
# 指定列名

pd.read_csv('Pandas/ex2.csv',header=None,names=['a', 'b', 'c', 'd', 'msg'])

,a,b,c,d,msg
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [58]:
# 指定行索引
pd.read_csv('Pandas/ex2.csv',header=None,names=['a', 'b', 'c', 'd', 'msg'],index_col='msg')

,a,b,c,d
msg,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [59]:
# 多层行索引

pd.read_csv('Pandas/ex2.csv',header=None,names=['a', 'b', 'c', 'd', 'msg'],index_col=['msg','a'])


,,b,c,d
msg,a,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


### 处理不规则的分隔符

In [60]:
%more Pandas/ex3.csv

In [61]:
# 正则表达式
pd.read_csv('Pandas/ex3.csv',sep='\s+')
# pd.read_table('Pandas/ex3.csv',sep=' ')

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


### 缺失值处理

In [62]:
%more Pandas/ex5.csv

In [63]:
pd.read_csv('Pandas/ex5.csv', na_values=['NA', 'NULL', 'foo'])


,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


In [64]:
pd.read_csv('Pandas/ex5.csv',na_values={'message':['foo','Na'],'something':['two']})

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


In [1]:
# 查看帮助文档
pd.read_table?


Object `pd.read_table` not found.


### 逐块读取数据


In [66]:
# 读10行
pd.read_csv('Pandas/ex6.csv',nrows=10)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


In [67]:
# 统计每个key出现次数
tr = pd.read_csv('Pandas/ex6.csv',chunksize=1000)
tr

In [68]:
key_count = pd.Series([])
for pieces in tr:
    key_count = key_count.add(pieces['key'].value_counts(),fill_value=0)
key_count = key_count.sort_values(ascending=False)
key_count[:10]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

## 保存到磁盘

In [69]:
df = pd.read_csv('Pandas/ex5.csv')
df

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [70]:
df.to_csv('Pandas/ex5_out.csv')
%more Pandas/ex5_out.csv

In [71]:
# 不写索引
df.to_csv('Pandas/ex5_out.csv', index=False)
%more Pandas/ex5_out.csv

In [72]:
# 不写列名称
df.to_csv('Pandas/ex5_out.csv', index=False,header=None)
%more Pandas/ex5_out.csv

In [73]:
# 指定分隔符
df.to_csv('Pandas/ex5_out.csv', index=False,header=None,sep='|')
%more Pandas/ex5_out.csv

In [74]:
# 只写出一部分列
df.to_csv('Pandas/ex5_out.csv', index=False,columns=['a','b'])
%more Pandas/ex5_out.csv


## 二进制格式

二进制的优点是容量小，读取速度快。缺点是可能在不同版本间不兼容。比如 Pandas 版本升级后，早期版本保存的二进制数据可能无法正确地读出来。

In [76]:
df = pd.read_csv('Pandas/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [79]:
pd.to_pickle(df,'Pandas/ex1_pickle.bin')
%ls Pandas

002001.csv                    ex6.csv
core_date_structure.ipynb     movie_lens_1m.ipynb
custom_date.csv               numpy.ipynb
data.csv                      pandas_groupby_and_agg.ipynb
dida365.csv                   pandas_intro.ipynb
ex1.csv                       pandas_intro_3.ipynb
ex1_pickle.bin                pandas_io.ipynb
ex2.csv                       pandas_re_index.ipynb
ex3.csv                       series_intro.ipynb
ex5.csv                       test1.ipynb
ex5_out.csv                   tips.csv


In [80]:
pd.read_pickle('Pandas/ex1_pickle.bin')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [82]:
pd.to_pickle(pd.read_csv('Pandas/ex1.csv'), 'Pandas/ex1_pickle.bin')
%ls Pandas

002001.csv                    ex6.csv
core_date_structure.ipynb     movie_lens_1m.ipynb
custom_date.csv               numpy.ipynb
data.csv                      pandas_groupby_and_agg.ipynb
dida365.csv                   pandas_intro.ipynb
ex1.csv                       pandas_intro_3.ipynb
ex1_pickle.bin                pandas_io.ipynb
ex2.csv                       pandas_re_index.ipynb
ex3.csv                       series_intro.ipynb
ex5.csv                       test1.ipynb
ex5_out.csv                   tips.csv


## 其他格式

* HDF5: HDF5是个C语言实现的库，可以高效地读取磁盘上的二进制存储的科学数据。
* Excel文件: pd.read_excel/pd.ExcelFile/pd.ExcelWriter
* JSON: 通过 json 模块转换为字典，再转换为 DataFrame
* SQL 数据库：通过 pd.io.sql 模块来从数据库读取数据
* NoSQL (MongoDB) 数据库：需要结合相应的数据库模块，如 pymongo 。再通过游标把数据读出来，转换为 DataFrame



